In [1]:
import torch
import torchvision
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




---


* resnet50 -> imagenet
* resnet50 -> cifar10
* resnet50 -> caltech


---



* MobileNet-V2 -> imagenet
* MobileNet-V2  -> cifar10
* MobileNet-V2 -> caltech


---

* VGG16 bn -> imagenet
* VGG16 bn -> cifar10
* VGG16 bn -> caltech

## resnet50 -> imagenet

In [ ]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
!tar -xvf "imagenette2-320.tgz"

Streaming output truncated to the last 5000 lines.
imagenette2-320/train/n03888257/n03888257_16077.JPEG
imagenette2-320/train/n03888257/n03888257_23339.JPEG
imagenette2-320/train/n03888257/n03888257_44204.JPEG
imagenette2-320/train/n03888257/n03888257_61633.JPEG
imagenette2-320/train/n03888257/n03888257_15067.JPEG
imagenette2-320/train/n03888257/n03888257_75365.JPEG
imagenette2-320/train/n03888257/n03888257_63966.JPEG
imagenette2-320/train/n03888257/n03888257_3927.JPEG
imagenette2-320/train/n03888257/n03888257_20684.JPEG
imagenette2-320/train/n03888257/ILSVRC2012_val_00047778.JPEG
imagenette2-320/train/n03888257/n03888257_14016.JPEG
imagenette2-320/train/n03888257/n03888257_37776.JPEG
imagenette2-320/train/n03888257/ILSVRC2012_val_00041706.JPEG
imagenette2-320/train/n03888257/n03888257_17513.JPEG
imagenette2-320/train/n03888257/n03888257_17143.JPEG
imagenette2-320/train/n03888257/n03888257_6738.JPEG
imagenette2-320/train/n03888257/n03888257_4355.JPEG
imagenette2-320/train/n03888257/ILS

In [ ]:
!unzip caltech-101.zip

unzip:  cannot find or open caltech-101.zip, caltech-101.zip.zip or caltech-101.zip.ZIP.


In [3]:
def getDataset(dataset):
    """
    CIFAR 10, ImageNet and Caltech dataset
    :return: return the dataset for using
    """
    DOWNLOAD = True
    if dataset == "cifar":
      transform=torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),torchvision.transforms.ToTensor()])
      _trainingSet = torchvision.datasets.CIFAR10(root=r"cifar", train=True, download=DOWNLOAD,transform=transform)
      _testingSet = torchvision.datasets.CIFAR10(root=r"cifar", train=False, download=DOWNLOAD,transform=transform)
      return _trainingSet, _testingSet

    elif dataset == "imagenet":
      transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
      _trainingSet = torchvision.datasets.ImageFolder(root=os.path.join("imagenette2-320", 'train'),transform=transform)
      _testingSet = torchvision.datasets.ImageFolder(root=os.path.join("imagenette2-320", 'train'),transform=transform)
      return _trainingSet, _testingSet

    elif dataset == "caltech":
      transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256, 256)),torchvision.transforms.ToTensor()])
      _trainingSet = torchvision.datasets.Caltech101(root=os.path.join("caltech-101"),transform=transform)
      _testingSet = torchvision.datasets.Caltech101(root=os.path.join("caltech-101"),transform=transform)
      return _trainingSet, _testingSet


resnet50 on imagenet

In [ ]:
resnet50 = torchvision.models.resnet50(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 90.1MB/s]


In [ ]:
torch.save(resnet50.state_dict(), '/content/drive/MyDrive/Colab Notebooks/MLCyberSec/PatchBackdoor/resnet50_imagenet.pth')

resnet50 on cifar

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from torchvision import transforms

class Scale(object):
    def __call__(self, image):
        return image / 255

# Combine custom scaling with other transforms
transform = transforms.Compose([
    torchvision.transforms.Resize(224),  # Resize for ResNet-50
    # Scale(),  # Scale pixel values
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='cifar', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
# Define ResNet-50 model and move it to GPU
model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    total = 0
    correct = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # if (i + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')

    epoch_accuracy = 100 * correct / total
    print(f'Accuracy of the model after epoch {epoch+1} is: {epoch_accuracy:.2f}%')


# Save the trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/MLCyberSec/PatchBackdoor/resnet50_cifar10.pth')


100%|██████████| 170498071/170498071 [00:01<00:00, 100004494.71it/s]


Extracting cifar/cifar-10-python.tar.gz to cifar


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Streaming output truncated to the last 5000 lines.
Epoch [4/10], Step [482/782], Loss: 1.1756622791290283
Epoch [4/10], Step [483/782], Loss: 1.2052359580993652
Epoch [4/10], Step [484/782], Loss: 1.1092281341552734
Epoch [4/10], Step [485/782], Loss: 1.0496562719345093
Epoch [4/10], Step [486/782], Loss: 1.0511901378631592
Epoch [4/10], Step [487/782], Loss: 1.1223646402359009
Epoch [4/10], Step [488/782], Loss: 1.2039690017700195
Epoch [4/10], Step [489/782], Loss: 1.0912361145019531
Epoch [4/10], Step [490/782], Loss: 1.2186014652252197
Epoch [4/10], Step [491/782], Loss: 1.0188215970993042
Epoch [4/10], Step [492/782], Loss: 1.1892234086990356
Epoch [4/10], Step [493/782], Loss: 0.9695834517478943
Epoch [4/10], Step [494/782], Loss: 1.044045329093933
Epoch [4/10], Step [495/782], Loss: 1.0061135292053223
Epoch [4/10], Step [496/782], Loss: 1.281278133392334
Epoch [4/10], Step [497/782], Loss: 0.8583031296730042
Epoch [4/10], Step [498/782], Loss: 1.167833685874939
Epoch [4/10], Ste

In [4]:
import logging
from typing import Callable, Union, Tuple
from typing import Callable, List, Optional, Dict, Union, Tuple
# needed for progress bar and time recording
import tqdm
import datetime
# pytorch
import torch
import torchvision.transforms
from torch.utils.data import DataLoader, Dataset
# save information when error
import traceback, atexit

In [5]:
def patchOnlyProtocol(pic: torch.Tensor, patch: torch.Tensor, resize: torchvision.transforms.Resize,
                      side: int) -> torch.Tensor:
    """
    apply the patch to the picture
    !!This is only a protocol,
    you should create a new function call who calls this and use that as the patchOnly function
    :param pic: the picture, expected 4D(B,C,H,W)
    :param patch: the patch, expected 3D(C,H,W)
    :param resize: the resize function
    :param side: the side of the patch
    :return: the picture
    """
    newX = torch.zeros(pic.shape, device=pic.device)
    newX[:] = patch
    newX[:, :, side:, side:] = resize(pic)
    return newX

In [6]:
def patchAndTriggerProtocol(pic: torch.Tensor, patch: torch.Tensor,
                            patchOnly: Callable[[torch.Tensor, torch.Tensor], torch.Tensor], trigger: torch.Tensor,
                            x: int, y: int):
    """
    apply the patch first then apply the trigger
    !!This is only a protocol,
    you should create a new function call who calls this and use that as the patchAndTrigger function

    :param pic: the picture, expected 4D(B,C,H,W)
    :param patch: the patch, expected 3D(C,H,W)\
    :param patchOnly: the apply patch function NOT THE PROTOCOL
    :param trigger: the trigger
    :param x: the top left corner x for trigger
    :param y: the top right corner y for trigger
    :return: the picture
    """
    patchedX = patchOnly(pic, patch)
    patchedX[:, :, x:x + trigger.shape[1], y:y + trigger.shape[2]] = trigger
    return patchedX

In [7]:
def getTransformations(picSize: int, patchSide: int, trigger: Union[int, torch.Tensor], device: str = "cuda") \
        -> Tuple[Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
                 Callable[[torch.Tensor, torch.Tensor], torch.Tensor]]:
    """
    a wrapper to get two tranformation needed for training
    !! everything should be a square!
    :param picSize: the side of image
    :param patchSide: the side of the patch
    :param trigger: the trigger size or the trigger itself
    :return: those two functions
    """
    resize = torchvision.transforms.Resize((picSize - patchSide, picSize - patchSide))

    def patchOnly(pic: torch.Tensor, patch: torch.Tensor) -> torch.Tensor:
        return patchOnlyProtocol(pic, patch, resize, patchSide)

    if type(trigger) == int:
        trigger = torch.ones(3, trigger, trigger, device=device)

    def patchAndTrigger(pic: torch.Tensor, patch: torch.Tensor) -> torch.Tensor:
        return patchAndTriggerProtocol(pic, patch, patchOnly, trigger, patchSide, patchSide)

    return patchOnly, patchAndTrigger

In [8]:
def codeBook(address: str) -> str:
    """
    get the code from the address and save them into dict with its key as its address and value as its code
    :param address: the address book
    :return: the code book [address,whole code]
    """
    with open(address, 'r', encoding='utf-8') as f:
        answer = f.read()
    return answer


In [9]:
def test(dataloader: Union[DataLoader, Tuple[Dataset, int]],
         models: Union[List[torch.nn.Module], torch.nn.Module],
         patch: torch.Tensor,
         transformation: List[Optional[Callable[[torch.tensor, torch.tensor], torch.tensor]]],
         norm: Optional[torchvision.transforms.Normalize],
         target: List[int],
         device: str = 'cuda', silence: bool = False) -> List[float]:
    """
    evaluate the given data loader with given data
    the patch is a parameter that parsed into the transformation like this:
    newX=transformation[i](x,patch)
    then we apply normalization:
    normedX=norm(x)
    :param dataloader: where the xs and ys comes from
                       it could be a dataloader, or a dataset with batchSize
    :param models: the model(s) we are about to evaluate
                   if it is a single model, then fine
                   if it is a list of models, then the length of it must match the length of transformation and target!
    :param patch: the patch we are about to evaluate 3D (C,H,W)
    :param transformation: a list of transformation that will be applied before the x was passed into the model.
        if None, then a dummy one (lambda x:x) would be used.
    :param norm: the normalization before calling the model
    :param target: a list of target class, None or negative numbers means use the original y.
    :param device: the device during evaluating
    :param silence: Do we show the process of testing?
    :return: a list of percentage of data equals to the target class
    """

    if norm is None:
        print("WARNING: No Normalization is passed in!")
        norm = lambda dummyX: dummyX

    if not issubclass(type(dataloader), DataLoader):
        # if a Dataset is passed in
        dataloader = DataLoader(dataloader[0], dataloader[1])

    if type(models) is not list:
        # in this case, models are just one model, we parse it to be a list of models
        models = [models for _ in range(len(target))]

    # keep track of original state of the models
    state = []
    for m in models:
        state.append(m.training)
        m.to(device)
        m.eval()
    del m

    if len(models) != len(transformation) or len(models) != len(target):
        raise Exception('The length of those should be same!')

    def realM(dummyX, dummyM):
        return dummyM(norm(dummyX))

    # if transformation is None, then simply return image (ignore the patch)
    transSize = len(transformation)
    for transI in range(len(transformation)):
        if transformation[transI] is None:
            transformation[transI] = lambda x, p: x
    del transI

    answer = [0 for _ in range(transSize)]

    if (silence):
        # if silence is on, then we do not need tqdm
        def myIter(dummyX):
            return dummyX
    else:
        # if silence is off, turn it on
        def myIter(dummyX):
            return tqdm.tqdm(iter(dummyX))
    del silence

    for x, y in myIter(dataloader):

        x = x.to(device)
        if (type(y) == torch.tensor):
            # if y is an integer, then we can not change the device of it
            y = y.to(device)

        for transI in range(transSize):
            currX = transformation[transI](x.clone(), patch)
            if (target[transI] == None or target[transI] < 0):
                # users could also use target[i]<0 to represent they want original label as the target
                answer[transI] += (torch.argmax(realM(currX, models[transI]), dim=1) ==
                                   y.to(device)).float().sum().item()
            else:
                answer[transI] += (torch.argmax(realM(currX, models[transI]), dim=1) ==
                                   torch.tensor(target[transI], device=device).repeat(y.shape)).float().sum().item()
    del x, y, transI, currX

    # dataset is assumed to have len
    dataSize = len(dataloader.dataset)
    for transI in range(transSize):
        answer[transI] /= dataSize

    # switch model states back
    for i in range(len(models)):
        m = models[i]
        if state[i]:
            m.train()
        else:
            m.eval()

    return answer

In [10]:
def train(name: str,
          trainLoader: Union[Dataset, DataLoader], valiLoader: Union[Dataset, DataLoader],
          models: Union[List[torch.nn.Module], torch.nn.Module], patch: torch.tensor,
          transformation: List[Callable[[torch.tensor, torch.tensor], torch.tensor]],
          norm: Optional[torchvision.transforms.Normalize],
          target: List[int],
          ratio: Optional[List[float]] = None, autoRatio: Optional[float] = None,
          batchSize: int = 64, lr: float = 0.001, rounds: int = 20,
          device: str = 'cuda',
          schedulerGamma: float = 0.8, schedulerMileStone: Optional[List[int]] = None,
          trainAccCheck: Union[bool, int] = False,
          valiAccCheck: Union[bool, int] = False,
          inProgressShow: bool = False,
          peak: bool = False, autosave: bool = True) -> Dict:
    """
    train a patch under a list of transformation and a list of target class
    :param name: the address of saving, also the name of the running
    Basic Inputs
    :param trainLoader: the Dataloader/Dataset for training
    :param valiLoader: the Dataloader/Dataset for testing set
    :param models: the model we are about to evaluate or a list of models we are about to evaulate.
                  In second case, please make sure the length of model, transfomation and target are same.
    :param patch: the patch we are about to train
    :param transformation: a list of transformations we want to apply to picture and patch
                           NOTE: for all the transformation in the list, it should be a function, it takes in pictures
                                 and the patch, and apply the patch on the pictures somehow.
                           EXAMPLE: x=transformation[0](x,patch)
    :param norm: the normalize function before we transfer x into models. If none, an identity function will be passed.
    :param target: the target class we try to approach to
                   NOTE: to refer the original label, use -1
                         to train without the original label, use -2
    :param ratio: the ratio to control the loss between different transformations
                  NOTE: default which is None which means same for everyone
                  EXAMPLE: [1,1.5] means the loss of the second transformation will be multiplied by 1.5
    :param autoRatio: if it is not None, the ratio of loss will be balanced.
                      For example, if autoRatio = 1, and the training loss is 30000 vs 20000,
                      then the ratio would be [1:1.5] on next round. if ratio is also set, they will be multiplied.
                      if auto ratio =0.8, in that case, the ratio would be[1:1+(1.5-1)*0.8]
    training setting
    :param batchSize: the batchSize for both Dataloader if Dataset is passed.
    :param lr: the learning rate for training
    :param rounds: how many epoch you want to train the patch
    :param device: the device for training
    scheduler setting
    :param schedulerGamma: the factor multiplied when scheduler milestone was achieved
    :param schedulerMileStone: the time we are about to change the milestone
    testing setting.
    If it is an int then it means after how many rounds we calculate something.
    :param trainAccCheck: do we check accuracy on training set.
    :param valiAccCheck: do we check accuracy on validation set
    :param inProgressShow: do we show the process during training
    other setting
    :param peak: true means save the picture after each transformation once
    :param autosave: save the result when it is stopped or finished
    :return: a dict of data, read by readData,
    """

    patch.requires_grad = True
    patch.to(device)

    if type(models) is not list:
        # in this case, models are just one model, we parse it to be a list of models
        models = [models for _ in range(len(target))]
    for m in models:
        m.eval()
        m.to(device)

    if len(models) != len(transformation) or len(models) != len(target):
        raise Exception('The length of those should be same!')

    optimizer = torch.optim.Adam([patch], lr=lr)

    if schedulerMileStone is None:
        schedulerMileStone = [20, 40, 60]
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=schedulerMileStone, gamma=schedulerGamma)

    soft = torch.nn.Softmax(dim=1)

    # This has to be "sum" since when calculating loss when target[i] is -2, we are doing it manually.
    lossCal = torch.nn.CrossEntropyLoss(reduction="sum")

    if issubclass(type(trainLoader), Dataset):
        trainCount = len(trainLoader)
        trainLoader = DataLoader(trainLoader, batchSize, shuffle=True, num_workers=8)
    else:
        trainCount = len(trainLoader.dataset)

    if issubclass(type(valiLoader), Dataset):
        valiCount = len(valiLoader)
        valiLoader = DataLoader(valiLoader, batchSize, shuffle=True, num_workers=8)
    else:
        valiCount = len(valiLoader.dataset)

    if norm is None:
        norm = lambda x: x

    # recording loss and accuracy
    transL = len(transformation)
    if ratio is None:
        ratio = [1 for _ in range(transL)]
    # record the ratio user want us to multiply, while ratio is the real ratio we are going to use next round
    baseRatio = ratio.copy()
    trainLossData = []
    trainAcc = []
    valiAcc = []
    timeData = []
    startTime = str(datetime.datetime.now())

    @atexit.register
    def save():
        endTime = str(datetime.datetime.now())
        report = {'patch': patch, 'model': [m.state_dict() for m in models],
                  'lr': lr, 'rounds': rounds-1, 'transCount': transL,
                  'train count': trainCount, 'vali count': valiCount,
                  'train loss': trainLossData,
                  'train acc': trainAcc, 'vali acc': valiAcc,
                  'start time': startTime, 'end time': endTime}
        # record the error reason
        stack = traceback.extract_stack()
        report['codebook'] = {}
        for i in range(0, len(stack) - 1):
            if 'CodeCAP' in stack[i].filename:
                report['codebook'][stack[i].filename] = codeBook(stack[i].filename)
        if autosave:
            torch.save(report, name + ".report")

    report = {'time': startTime,
              'info': 'The program has started running. Please wait until the first round finish to make sure it runs correctly.'
                      ' If program exit on error after the first round complete, information will be saved here.'}
    if autosave:
        torch.save(report, name + ".report")

    if (peak):
        x = trainLoader.dataset[0][0]
        x = x.unsqueeze(0)
        x = x.to(device)
        from torchvision.utils import save_image
        save_image(x, name + "base.png")
        for transI in range(transL):
            save_image(transformation[transI](x, patch), name + str(transI) + ".png")

    def realTest(dataLoader, dummyPatch):
        return test(dataLoader, models, dummyPatch.clone(), transformation, norm, target, device, silence=True)

    def realM(dummyX, dummyM):
        return dummyM(norm(dummyX))

    print("START:" + str(realTest(valiLoader, patch)))
    tqdmIter = tqdm.tqdm(range(rounds))
    for roundI in tqdmIter:
        # start training
        trainLoss = [0 for _ in range(transL)]
        valiLoss = [0 for _ in range(transL)]
        for x, y in trainLoader:
            x = x.to(device)
            y = y.to(device)
            loss = 0
            for transI in range(transL):
                currX = x.clone()
                transedX = transformation[transI](currX, patch)
                currO = realM(transedX, models[transI])
                if (target[transI] == -1):
                    # target[i]=-1 means they want to train with given Y
                    currLoss = lossCal(currO, y)
                elif (target[transI] == -2):
                    o = realM(x.clone(), models[transI])
                    # give a minimum bound on value to avoid nan
                    currLoss = -(soft(o) * torch.log(torch.clamp(soft(currO), min=2 ** -149))).sum()
                    del o
                elif (target[transI] >= 0):
                    # otherwise, use the given target
                    currLoss = lossCal(currO, torch.stack([torch.tensor(target[transI], device=device)] * x.shape[0]))
                else:
                    raise ("Not implemented yet")
                loss += currLoss * ratio[transI]
                trainLoss[transI] += currLoss.detach().cpu().item()
            # Step
            optimizer.zero_grad()
            # certainly, loss could be backward
            loss.backward()
            optimizer.step()
            # make sure the patch is legal
            with torch.no_grad():
                patch[:] = torch.clamp(patch, 0, 1)
        del x, y, currX, currO,
        scheduler.step()
        if autoRatio is not None:
            minLoss = min(trainLoss)
            lossRatio = trainLoss.copy()
            for i in range(transL):
                lossRatio[i] /= minLoss
                lossRatio[i] = 1 + (lossRatio[i] - 1) * autoRatio
                ratio[i] = baseRatio[i] * lossRatio[i]

        if (trainAccCheck is True or (trainAccCheck > 0 and roundI % trainAccCheck == 0)):
            currTrainAcc = realTest(trainLoader, patch)
        else:
            currTrainAcc = [0 for _ in range(transL)]

        if (valiAccCheck is True or (valiAccCheck > 0 and roundI % valiAccCheck == 0)):
            currValiAcc = realTest(valiLoader, patch)
        else:
            currValiAcc = [0 for _ in range(transL)]

        if (inProgressShow):
            print("train loss: ")
            trainLossSum = 0
            for trainLossAnon in trainLoss:
                print("%.3f; " % (trainLossAnon), end="")
                trainLossSum += trainLossAnon
            print("sumLoss: %.3f" % trainLossSum)
            del trainLossAnon, trainLossSum

            print("train Top1: ")
            for trainAccAnon in currTrainAcc:
                print("%.2f;  " % (trainAccAnon), end="")
            del trainAccAnon
            print()

            print("vali Top1: ")
            for valiAccAnon in currValiAcc:
                print("%.2f;  " % (valiAccAnon), end="")
            del valiAccAnon
            print()

            if autoRatio is not None:
                print("raio:")
                for ratioAnon in ratio:
                    print("%.2f;  " % (ratioAnon), end="")
                print()
        trainLossData.append(trainLoss)
        trainAcc.append(currTrainAcc)
        valiAcc.append(currValiAcc)
        timeData.append(tqdmIter.format_dict['elapsed'])

    print("END:" + str(realTest(valiLoader, patch)))
    # we assume Dataset has length
    report = {'patch': patch, 'model': 'models', 'lr': lr, 'rounds': rounds,
              'transCount': transL, 'train count': trainCount, 'vali count': valiCount,
              'train loss': trainLossData,
              'train acc': trainAcc, 'vali acc': valiAcc,
              'time': timeData}
    stack = traceback.extract_stack()
    report['codebook'] = {}
    for i in range(0, len(stack) - 1):
        report['codebook'][stack[i].filename] = codeBook(stack[i].filename)
    if autosave:
        torch.save(report, name + ".report")
    return report

In [11]:
import gc
torch.cuda.empty_cache()
gc.collect()

30

In [12]:
logging.basicConfig(level=logging.INFO)
trainSet, testSet = getDataset('cifar')
if torch.cuda.is_available():
    device = 'cuda'
else:
    logging.warning('The code is suggested to run in CUDA. No CUDA detected')
    device = 'cpu'
side = 40
size = 40
patchOnly, patchAndTrigger = getTransformations(224, side, size)
m = torchvision.models.resnet50(num_classes=10)
anonM = torch.load(r'/content/drive/MyDrive/Colab Notebooks/MLCyberSec/PatchBackdoor/resnet50_cifar10.pth')
m.load_state_dict(anonM)

trans = torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261])
patch = torch.zeros(3, 224, 224, device=device)

train('result', trainSet, testSet, m, patch,
                transformation=[patchOnly, patchAndTrigger],
                norm=trans,
                target=[-2, 9], inProgressShow=True, trainAccCheck=True, valiAccCheck=True,
                rounds=10,
                batchSize=32,device=device,autoRatio=0.5)

100%|██████████| 170498071/170498071 [00:06<00:00, 28411632.82it/s]


Extracting cifar/cifar-10-python.tar.gz to cifar
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - P

START:[0.5931, 0.246]


 10%|█         | 1/10 [28:43<4:18:28, 1723.16s/it]

...

train loss: 
43167.798; 17247.847; sumLoss: 60415.645
train Top1: 
0.68;  0.94;  
vali Top1: 
0.66;  0.94;  
raio:
1.75;  1.00;  


100%|██████████| 10/10 [4:47:45<00:00, 1726.57s/it]

train loss: 
43048.948; 17069.423; sumLoss: 60118.371
train Top1: 
0.69;  0.92;  
vali Top1: 
0.67;  0.92;  
raio:
1.76;  1.00;  


END:[0.6728, 0.9219]


FileNotFoundError: ignored